<a href="https://colab.research.google.com/github/trtrgfh/GlVYfAbQjtuwUHlZ/blob/main/Potential_Talents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/content/potential-talents.csv")
print(df.shape)
df.head()

(104, 5)


,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [3]:
job_titles = df["job_title"].to_list()
job_titles = [string.lower() for string in job_titles]
keywords = ["aspiring human resources", "seeking human resources"]

combined_list = keywords + job_titles

## Preprocessing

## Bag-of-words

In [4]:
from nltk.stem import PorterStemmer        

In [5]:
CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
# fit and transform
Count_data = CountVec.fit_transform(combined_list)

cv_keyword_vec = Count_data[:len(keywords)].toarray()
cv_title_vec = Count_data[len(keywords):].toarray()

In [6]:
# Compute cosine similarity between the keyword vectors and job title vectors
cv_cs = cosine_similarity(cv_keyword_vec, cv_title_vec)

In [7]:
# Get the cos similarity of each candidata with reaspect to candidata id
cv_res = pd.DataFrame(cv_cs,columns=df['id'])

In [8]:
cv_res_key1 = cv_res.iloc[0]
cv_res_key2 = cv_res.iloc[1]
tfidf_res1 = pd.DataFrame(cv_res_key1, index=df['id'])
tfidf_res2 = pd.DataFrame(cv_res_key2, index=df['id'])
print(cv_res.shape)

(2, 104)


In [9]:
cv_res_key1

id
1      0.500000
2      0.000000
3      0.866025
4      0.000000
5      0.000000
         ...   
100    0.680414
101    0.577350
102    0.000000
103    0.000000
104    0.000000
Name: 0, Length: 104, dtype: float64

In [10]:
cv_res_key2

id
1      0.333333
2      0.000000
3      0.577350
4      0.000000
5      0.000000
         ...   
100    0.680414
101    0.577350
102    0.000000
103    0.000000
104    0.000000
Name: 1, Length: 104, dtype: float64

## Tf-Idf vectorizer

In [11]:
tfidf_vec = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english')
 
tfidf_data = tfidf_vec.fit_transform(combined_list)

tfidf_keyword_vec = tfidf_data[:len(keywords)].toarray()
tfidf_title_vec = tfidf_data[len(keywords):].toarray()

In [12]:
tfidf_cs = cosine_similarity(tfidf_keyword_vec, tfidf_title_vec)

In [13]:
tfidf_res = pd.DataFrame(cv_cs,columns=df['id'])

In [14]:
tfidf_res_key1 = tfidf_res.iloc[0]
tfidf_res_key2 = tfidf_res.iloc[1]
tfidf_res1 = pd.DataFrame(tfidf_res_key1, index=df['id'])
tfidf_res2 = pd.DataFrame(tfidf_res_key2, index=df['id'])
print(tfidf_res.shape)

(2, 104)


In [15]:
tfidf_res1

,0
id,
1,0.500000
2,0.000000
3,0.866025
4,0.000000
5,0.000000
...,...
100,0.680414
101,0.577350
102,0.000000


In [16]:
tfidf_res2

,1
id,
1,0.333333
2,0.000000
3,0.577350
4,0.000000
5,0.000000
...,...
100,0.680414
101,0.577350
102,0.000000


## Word2vec

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v
from sklearn.decomposition import PCA

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
sw = stopwords.words('english')

In [19]:
# Tokenize each job title
word2vec_tokens = [word_tokenize(job_title) for job_title in job_titles]

In [20]:
# Remove stop words and puntuations
word2vec_clean = []

for line in word2vec_tokens: 
  tokens = []
  for word in line:
    if (word not in sw and word not in string.punctuation):  
        tokens.append(word)
  word2vec_clean.append(tokens)

In [21]:
w = w2v(
    word2vec_clean,
    min_count=1,  
    sg = 1,       
    window=5      
)       

# Create word embedding 
word2vec_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index = w.wv.key_to_index
    )
)
print(word2vec_df.shape)
word2vec_df.head()

(185, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
human,-0.000721,0.001137,0.004927,0.009055,-0.008244,-0.008943,0.007436,0.010913,-0.005582,-0.004577,...,0.003089,0.001556,0.003665,0.001883,0.011729,0.006008,-0.008459,-0.007420,0.001335,0.005793
resources,-0.008932,0.004368,0.005016,0.005808,0.008211,-0.007896,0.001784,0.007625,-0.003179,-0.006778,...,0.002101,-0.000378,0.002670,-0.006512,-0.000931,0.003528,0.005826,-0.002526,-0.009151,0.004093
aspiring,0.000145,0.003494,-0.007041,-0.001435,0.008183,0.006441,-0.003321,0.003717,-0.008738,0.005776,...,-0.003684,0.006493,0.009174,-0.003180,0.008934,0.005942,0.006044,0.000199,0.008570,-0.007339
professional,-0.008349,0.009518,-0.000155,-0.001794,0.004900,-0.004827,0.003044,0.007707,0.005900,-0.007954,...,-0.006945,-0.000452,-0.000588,-0.001859,0.010418,-0.000043,0.006028,-0.007580,-0.002253,-0.005721
student,-0.007151,0.001511,-0.007245,-0.002277,0.003972,0.005233,0.001529,0.002735,-0.004286,0.006964,...,0.003588,-0.004248,0.005417,-0.003617,0.003296,-0.007846,0.006414,0.004797,0.000978,0.002826


In [22]:
word2vec_keyword_vec1 = [w.wv.get_vector(keyword) for keyword in ["aspiring", "human", "resources"]]
word2vec_keyword_vec2 = [w.wv.get_vector(keyword) for keyword in ["seeking", "human", "resources"]]
word2vec_cs1 = []
word2vec_cs2 = []
for title in word2vec_clean:
  word2vec_cs1.append(cosine_similarity(word2vec_keyword_vec1, [w.wv.get_vector(word) for word in title]).mean())
  word2vec_cs2.append(cosine_similarity(word2vec_keyword_vec2, [w.wv.get_vector(word) for word in title]).mean())

In [23]:
word2vec_res1 = pd.DataFrame(word2vec_cs1, index=df['id'])
word2vec_res2 = pd.DataFrame(word2vec_cs2, index=df['id'])

In [24]:
word2vec_res1

,0
id,
1,0.123920
2,0.068883
3,0.263165
4,0.011653
5,0.054677
...,...
100,0.148578
101,0.190823
102,-0.006135


In [25]:
word2vec_res2

,0
id,
1,0.089942
2,0.050999
3,0.197722
4,0.030509
5,0.050008
...,...
100,0.146592
101,0.173113
102,0.012889


## Glove

In [26]:
from gensim.models import KeyedVectors

glove_embeddings = {}
# Load the pre-trained GloVe word vectors
with open("/content/glove.6B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      glove_embeddings[word] = vector

In [27]:
sw = stopwords.words('english')
# Tokenize each job title
glove_tokens = [word_tokenize(job_title) for job_title in job_titles]
# Remove stop words and puntuations
glove_clean = []

for line in word2vec_tokens: 
  tokens = []
  for word in line:
    if (word not in sw and word not in string.punctuation):  
        tokens.append(word)
  glove_clean.append(tokens)

In [ ]:
glove_keyword_vec1 = [glove_embeddings[keyword] for keyword in ["aspiring", "human", "resources"]]
glove_keyword_vec2 = [glove_embeddings[keyword] for keyword in ["seeking", "human", "resources"]]
glove_cs1 = []
glove_cs2 = []
for title in glove_clean:
  glove_title_vec = [glove_embeddings[word] for word in title if word in glove_embeddings]
  glove_cs1.append(cosine_similarity(glove_keyword_vec1, glove_title_vec).mean())
  glove_cs2.append(cosine_similarity(glove_keyword_vec2, glove_title_vec).mean())

In [ ]:
glove_res1 = pd.DataFrame(glove_cs1, index=df['id'])
glove_res2 = pd.DataFrame(glove_cs2, index=df['id'])

In [ ]:
glove_res1

In [ ]:
glove_res2

## Bert

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def get_bert_embeddings(text):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Convert tokens to token IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Convert token IDs to tensors
    input_tensor = torch.tensor([token_ids])

    # Get the BERT model outputs
    with torch.no_grad():
        outputs = model(input_tensor)

    # Get the word embeddings
    embeddings = outputs.last_hidden_state.squeeze()

    return embeddings

In [ ]:
# Get word embeddings
bert_embeddings = []

# Iterate over each string
for title in job_titles:
    # Append word embeddings to the list
    bert_embeddings.append(get_bert_embeddings(title))

In [ ]:
# Get keyword embeddings
keyword1 = "aspiring human resources"
keyword2 = "seeking human resources"

bert_key_embeddings1 = get_bert_embeddings(keyword1)
bert_key_embeddings2 = get_bert_embeddings(keyword2)

# Print the shape of the word embeddings
print(bert_key_embeddings1.shape)
print(bert_key_embeddings2.shape)

In [ ]:
# bert_embeddings is in shape (examples, # of words, # of features)
bert_cs1 = []
bert_cs2 = []
for title_emb in bert_embeddings:
  if title_emb.ndim != 2:
    title_emb = title_emb.reshape(1, -1)
  bert_cs1.append(cosine_similarity(bert_key_embeddings1, title_emb).mean())
  bert_cs2.append(cosine_similarity(bert_key_embeddings2, title_emb).mean())

In [ ]:
bert_res1 = pd.DataFrame(bert_cs1, index=df['id'])
bert_res2 = pd.DataFrame(bert_cs2, index=df['id'])

In [ ]:
bert_res1

In [ ]:
bert_res2

## Sbert

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the SBERT model
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode the sentences to obtain embeddings
sbert_embeddings = sbert_model.encode(job_titles)
print(sbert_embeddings.shape)

In [ ]:
sbert_keyword_vec1 = model.encode("aspiring human resources").reshape(1, -1)
sbert_keyword_vec2 = model.encode("seeking human resources").reshape(1, -1)
sbert_cs1 = []
sbert_cs2 = []
for embedding in sbert_embeddings:
  embedding = embedding.reshape(1, -1)
  sbert_cs1.append(cosine_similarity(sbert_keyword_vec1, embedding).mean())
  sbert_cs2.append(cosine_similarity(sbert_keyword_vec2, embedding).mean())

In [ ]:
sbert_res1 = pd.DataFrame(sbert_cs1, index=df['id'])
sbert_res2 = pd.DataFrame(sbert_cs2, index=df['id'])

In [ ]:
sbert_res1

In [ ]:
sbert_res2